<a href="https://colab.research.google.com/github/alienlifeform/exotic-colab/blob/main/saved_colabs/exo-refactor2_EXOTIC_See_it_in_Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXOTIC: See it in Action**

<font size=2>
Following is a demonstration of how to run EXOTIC's code in your browser on a set of sample telescope images to generate a lightcurve.
</font>

<font size=2>Steps:</font>

<ol><font size=2><li>Load telescope images
 <ul><font size=2><li>For this tutorial, we'll use images taken from a previous transition of planet HAT-P-32 b</li></font></ul></li>
<li>Identify target and comparison stars in a telescope image
 <ul><font size=2><li>Find the target star (HAT-P-32) and several comparison stars that have stable brightness</li></font>
 <font size=2><li>Enter their coordinates to calibrate the baseline brightness</li></font></ul></li>
<li>Run EXOTIC to generate a lightcurve
 <ul><font size=2><li>Watch EXOTIC determine if the brightness of your target star changes, signifying a transit. A transit is when an exoplanet passes in front of its host star, momentarily dimming out the star's light. <a href="https://exoplanets.nasa.gov/alien-worlds/ways-to-find-a-planet/#/2">(See an animation of a transit)</a></li></font></ul></li></font></ol>



<font size=2>INSTRUCTIONS</font>

<font size=2>

Click the Screen Shot 2022-09-25 at 2.30.43 PM.png icons below to run the code for each step. Wait for it to finish running before moving to the next step, and watch for the output describing what is happening.
</font>

<br />
<table><tr>
  <td cellpadding=0 cellspacing=0 bgcolor="#fee9e9">
  <font size=2 color="#c02020">Important:</font> During this tutorial, it is recommended that you do not click the blue "<font color="#5070e0">Show code</font>" links.</td>
</tr></table>
<br />

---

## Step 1: Load Telescope Images

<font size=2>
🕔 Estimated time: 2 minutes

For this tutorial, we are using sample images from a known transit. When you run it yourself, you will upload your own images.
</font>

<br />
<table><tr><td bgcolor="#fee9e9">
  <font size=2 color="#c02020">Important:</font>
  <ul><li><font size=2>A popup will appear titled "Warning: This notebook was not authored by Google.". Read it, then click "Run Anyway".</font></li>
  <li><font size=2>Ignore any warning that may ask you to "RESTART RUNTIME"</font></li></ul>
</td></tr></table>
<br />

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>Load telescope images</font>

##############################################################
%%capture step_capture --no-display

from IPython.display import display, HTML, Javascript
display(HTML('<p class="bookend">START: Importing necessary software libraries</p>'))
display(HTML('<p class="hidden">Loading styles, please wait...</p>'))

# Install EXOTIC Colab interface code
!pip install git+https://github.com/alienlifeform/exotic-colab.git --upgrade

from exoticcolab.display import setupDisplay, testImplementation, displayStep, makeContainer, downloadButton, appendToContainer, appendStepToContainer, expandableSection, hideWarning, showProgress, resize_colab_cell

# Set up custom Colab styles and interactions
setupDisplay()
# Improve how colab handles long code output fields
get_ipython().events.register('pre_run_cell', resize_colab_cell)


display(HTML('<ul class="step_container_1a"></ul>'))
appendStepToContainer('.step_container_1a','Styles loaded, importing libraries...')

appendStepToContainer('.step_container_1a','(1/5) Bokeh.io')
import bokeh.io
from bokeh.io import output_notebook

appendStepToContainer('.step_container_1a','(2/5) EXOTIC <span class="comment">(This will take up to a minute, please wait...)</span>')

#!pip install exotic --upgrade
!pip install git+https://github.com/alienlifeform/exotic-prototype.git --upgrade
from exotic.api.colab import *
# plot_image not needed for the simulation
# from exotic.api.plotting import plot_image 

# This suppresses the "RESTART RUNTIME" button
hideWarning()
display(HTML('<br /><p>If there is a "RESTART RUNTIME" warning button above, you can ignore it (or you\'ll have to re-run this step).</p>'))

appendStepToContainer('.step_container_1a','(3/5) NASAExoplanetArchive, Astropy, Utils')
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target
from IPython.display import Image
from ipywidgets import widgets, HBox
import os
import re
import json

appendStepToContainer('.step_container_1a','(4/5) Matlab, SciPy')

appendStepToContainer('.step_container_1a','(5/5) Google Utils')
from google.colab import files

#expandableSection('<p>This is some expandable stuff</p>')
display(HTML('<p class="bookend">DONE: Importing necessary software libraries</p>'))


#########################################################

display(HTML('<p class="bookend">START: Loading telescope images</p>'))
display(HTML('<ul class="step_container_1b"></ul>'))

appendStepToContainer('.step_container_1b','Ensuring sample images are loaded...')

#
# Delete existing sample data by changing to `rebuild = "true"`
#
rebuild = "true"  
if rebuild == "true":
  if os.path.isdir("/content/EXOTIC/exotic-in-action"):
    %rm -rf /content/EXOTIC/exotic-in-action
    display(HTML('<p class="step">"Rebuild" flag is "true"... Removing old images at /content/EXOTIC/exotic-in-action</p>'))

#
# Download sample files if necessary
#
#sample_data_source = "https://github.com/rzellem/EXOTIC_sampledata.git" # goes into sample_data_target_folder
sample_data_source = "https://github.com/alienlifeform/exotic-sample-data.git" # goes into sample_data_target_parent
sample_data_target_parent = "/content/EXOTIC/exotic-in-action"
sample_data_target_folder = "/content/EXOTIC/exotic-in-action/sample-data"
sample_data_target_child = "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017"
sample_data_target_outputs = "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output"
if os.path.isdir(sample_data_target_child):
  appendStepToContainer('.step_container_1b','Skipping... Sample images already loaded')

else:
  appendStepToContainer('.step_container_1b','Downloading images from ' + sample_data_source)
  #git_rv = !git clone {sample_data_source} {sample_data_target_folder}
  git_rv = !git clone {sample_data_source} {sample_data_target_parent}
  git_co_rv = !cd {sample_data_target_parent} && !git checkout beta1
  appendStepToContainer('.step_container_1b','Telescope images successfully loaded for HAT-P-32 b')

#
# Show files found
#
#!du -hd0 --exclude ".*" /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017
numfiles_fits = !ls {sample_data_target_child} | grep -ci FITS
numfiles_json = !ls {sample_data_target_child} | grep -ci json

display(HTML('<p class="step">You have ' + str(numfiles_fits[0]) + ' telescope image (.FITS) files.</p>'))
#display(HTML('<p class="step">You have ' + str(numfiles_json[0]) + ' inits (.json) files</p>'))

display(HTML('<p class="bookend">DONE: Loading telescope images. <b>You may move on to the next step.</b></p>'))

---

## Step 2: Identify target star and comparison stars

<font size=2>
🕔 Estimated time: 4 minutes

In this multi-part step, you will use your stargazing skills to identify coordinates for your target star (HAT-P-32). You will:
</font>


<ol type="a"><font size=2><li>Watch the Video: "How to determine a star's coordinates" (2 minutes)</li>
<li>Try it yourself: Enter the coordinates for the target and comparison stars (2 minutes)</li></font></ol>


In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>2a. Watch the video: "How to enter a star's coordinates"</font>

setupDisplay()

display(HTML('''
<br />
<video width="840" controls="true" poster="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.png" src="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.mov"/>
'''))

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>2b. Identify the target and comparison stars in a telescope image</font>

setupDisplay()

#########################################################
#from exotic.api.colab import *

display(HTML('<p class="bookend">START: Compare telescope image and star chart</p>'))

display(HTML('<ul class="step4">'))

bokeh.io.output_notebook()
sample_data = False

p = "sample-data/HatP32Dec202017"
p = check_dir(os.path.join("/content/EXOTIC/exotic-in-action/", p))
output_dir = os.path.join(p, "EXOTIC_output/")      
                                         
inits = []    # array of paths to any inits files found in the directory

all_files_print = [f for f in sorted(os.listdir(p))]
all_files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
for f in all_files:
  if re.search(r"\.f[itz]+s?\.?g?z?$", f, re.IGNORECASE):
    if first_image == "":
      first_image = os.path.join(p, f)
    fits_count += 1
  if re.search(r"\.json$", f, re.IGNORECASE):
    inits.append(os.path.join(p, f))

#print('Found ' + str(fits_count) + ' image files and ' + str(len(inits)) + ' initialization files in the directory')

if fits_count >= 2:
  obs = ""
  display(HTML('''
  <h3>Data Entry 1 of 2: Enter coordinates for the target star</h3>
  <ol class="step">
  <li class="step4">In the right image, find the <i>crosshairs</i> in the center - that represents your target star.</li>
  <li class="step4">On the left image, <i>find this target star and roll over it with your mouse</i>, note the X and Y coordinates.</li>
  <li class="step4">Put the X and Y coordinates in the box below in the format <code>[x,y]</code> and press return.</li></ol>
  <p>Tip: Use the zoom feature. Click the magnifying glass and click-and-drag to draw a rectangle that matches the starchart.</p>

  <br /><br />
  <div class="plots">
  <img align=right src="https://app.aavso.org/vsp/chart/X28194FDL.png" width=380>
  '''))
  display_image(first_image)
  display(HTML('</div><br clear="all"/>'))

  # verify the entry is good enough
  success = False
  while not success:
    targ_coords = input("Enter coordinates for target star - [424,286] - and press return:  ")  

    # check syntax and coords
    tc_syntax = re.search(r"\[\d+,\d+\]$", targ_coords)
    if tc_syntax:
      #display(HTML('<p class="output">Syntax OK: [x,y] e.g. ' + targ_coords + '</p>'))
      tc_coords = re.findall("\d+", targ_coords)
      if 422 <= int(tc_coords[0]) <= 426 and 284 <= int(tc_coords[1]) <= 288:
        #display(HTML('<p class="output">Coordinates OK: ' + targ_coords + ' are close to [424,286]</p>'))
        success = True
      else:
        display(HTML('<p class="output error">Try again, your coordinates are a bit off: ' + targ_coords + ' should be closer to [424,286]</p>'))
    else:
      display(HTML('<p class="output error">Try again, your syntax is not quite right: ' + targ_coords + ' needs to look like [424,286]</p>'))

  # if targ_coords != "[424,286]":
  #   display(HTML('<p class="output">You entered ' + targ_coords + '</p>'))
  # targ_coords = "[424,286]"


  showProgress(1)
  display(HTML('<p class="output">Target star coordinates saved to inits.json</p>'))
  
  display(HTML('<h3>Data Entry 2 of 2: Enter coordinates for at least two comparison stars.</h3>'))
  display(HTML('<ol class="step"><li class="step4">In the right image, find the stars <i>with numbers</i> that represent suggested comparison stars.</li><li class="step4">On the left image, <i>find and roll over each comparison star with your mouse</i> and note the coordinates.</li><li class="step4">Put the X and Y coordinates in the box below in the format <code>[[x1,y1][x2,y2]]</code> and press return.</li></ol>'))
  
  # verify the entry is good enough
  success = False
  while not success:
    comp_coords = input("Enter coordinates for the comparison stars - [[326,365],[416,343]] - and press return:  ")  

    # check syntax
    cc_syntax = re.search(r"\[(\[\d+,\d+\],?)+\]$", comp_coords)
    if cc_syntax:
      #display(HTML('<p class="output">Syntax OK:  [[x1,y1],[x2,y2]] e.g. ' + comp_coords + '</p>'))
      success = True
    else:
      display(HTML('<p class="output error">Try again, your syntax is not quite right: ' + comp_coords + ' needs to look more like [[326,365],[416,343]]</p>'))

  # if comp_coords != "[[326,365],[416,343],[491,303]]":
  #   display(HTML('<p class="step">You entered ' + comp_coords + '<br /><br /></p>'))
  # comp_coords = "[[326,365],[416,343],[491,303]]"

  showProgress(1)
  display(HTML('<p class="output">Comparison star coordinates saved to inits.json</p>'))
  display(HTML('<p class="output">Images and inits.json set up. Ready for EXOTIC data reduction/analysis</p>'))
  aavso_obs_code = ""

  if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

  #inits = [make_inits_file(planetary_params, p, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data)]
  inits = [output_dir+"inits.json"]

  if not os.path.isdir(output_dir):    # Make the output directory if it does not exist already.
    os.mkdir(output_dir)
  output_dir_for_shell = output_dir.replace(" ", "\ ")

#print("Path to the inits file(s) that will be used:")

#for inits_file in inits:
#  print(inits_file)

num_inits = len(inits)


display(HTML('<p class="bookend">DONE: Compare telescope image and star chart. <b>You may move on to the next step.</b></p>'))




---

## Step 3: Run EXOTIC to generate a lightcurve

<font size=2>
🕔 Estimated time: 1 minute (this would take longer with a real data set ~5-15mins)

EXOTIC will analyze the brightness of the target star in each telescope image to identify if there was a transit. At the end of this step, you will see your lightcurve as an image, and you can download the lightcurve data to your hard drive for submission to AAVSO.
</font>

<br />
<table><tr>
  <td cellpadding=0 cellspacing=0 bgcolor="#fee9e9">
  <font size=2 color="#c02020">Important:</font> <font size=2>Do not actually submit the sample lightcurve from this tutorial to AAVSO.</font></td>
</tr></table>
<br />

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>Run EXOTIC to generate a lightcurve</font>

# This generation is a simulation only using unstyled logging from a HAT-P-32 b EXOTIC run

setupDisplay()

display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))

log_10 = """
Path to the inits file(s) that will be used:
/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json
!exotic -red "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json" -ov
    Downloading __databases__.pickle_new...
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 1.10.2
*************************************************************


**************************
Complete Reduction Routine
**************************

**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....

"""

log_20 = """
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/wcs.fits

Checking for variability in Comparison Star #1:
    Pixel X: 493 Pixel Y: 304
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Checking for variability in Comparison Star #2:
    Pixel X: 415 Pixel Y: 343
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Computing best comparison star, aperture, and sky annulus. Please wait.
"""

log_30 = """

*********************************************
Best Comparison Star: None
Minimum Residual Scatter: 0.672%
Optimal Aperture: 5.02
Optimal Annulus: 7.54
*********************************************

No Barycentric Julian Dates (BJDs) in Image Headers for standardizing time format. Converting all JDs to BJD_TDBs.
Please be patient- this step can take a few minutes.
"""

log_40 = """
Thinking ... DONE!


Output File Saved


****************************************
Fitting a Light Curve Model to Your Data
****************************************

[ultranest] Sampling 400 live points from prior ...


Mono-modal Volume: ~exp(-3.81) * Expected Volume: exp(0.00) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|**********|     +3.0

Z=-1659103.7(0.00%) | Like=-1655784.03..-111.41 [-8987932.8990..-176103.1770] | it/evals=80/494 eff=85.1064% N=400 

Mono-modal Volume: ~exp(-3.96) * Expected Volume: exp(-0.23) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|********  |     +3.0

Z=-338377.3(0.00%) | Like=-334422.26..-111.41 [-8987932.8990..-176103.1770] | it/evals=160/609 eff=76.5550% N=400 

Mono-modal Volume: ~exp(-4.18) * Expected Volume: exp(-0.45) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|*******   |     +3.0

Z=-202573.2(0.00%) | Like=-201882.30..-111.41 [-8987932.8990..-176103.1770] | it/evals=240/730 eff=72.7273% N=400 

"""

log_50 = """
Mono-modal Volume: ~exp(-20.47)   Expected Volume: exp(-17.10) Quality: ok

rprs:    +0.000|        * |   +0.186
tmid: +2458107.706|       ** |+2458107.716
ars :      +4.8|    **    |     +5.9
a2  :    -1.000|    *     |   +1.000

[ultranest] Explored until L=-7e+01  
[ultranest] Likelihood function evaluations: 15883
[ultranest]   logZ = -82.92 +- 0.141
[ultranest] Effective samples strategy satisfied (ESS = 2176.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.14, need <0.5)
[ultranest]   logZ error budget: single: 0.16 bs:0.14 tail:0.01 total:0.14 required:<0.50
[ultranest] done iterating.

*********************************************************
FINAL PLANETARY PARAMETERS

          Mid-Transit Time [BJD_TDB]: 2458107.7136 +/- 0.001
  Radius Ratio (Planet/Star) [Rp/Rs]: 0.1587 +/- 0.0037
           Transit depth [(Rp/Rs)^2]: 2.52 +/- 0.12 [%]
 Semi Major Axis/ Star Radius [a/Rs]: 5.36 +/- 0.11
               Airmass coefficient 1: 1.1645 +/- 0.0078
               Airmass coefficient 2: -0.1222 +/- 0.0028
                    Residual scatter: 0.67 %
                 Best Comparison Star: None
                    Optimal Aperture: 5.02
                     Optimal Annulus: 7.54
              Transit Duration [day]: 0.1311 +/- 0.0028
*********************************************************
Output Files Saved

************************
End of Reduction Process
************************
"""

log_60 = """
************************
EXOTIC has successfully run!!!
It is now safe to close this window.
************************
lightcurve: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/FinalLightCurve_HAT-P-32 b_2017-12-19.png
"""

import time
print(log_10)
showProgress(1)
print(log_20)
showProgress(2)
print(log_30)
showProgress(1)
print(log_40)
showProgress(2)
print(log_50)
showProgress(1)
print(log_60)

display(HTML('''
    <img width=800 src="https://exoplanets.nasa.gov/system/exotic/sample_lightcurve.png">

    <p><a href="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.mov">Click for a video to help you understand this lightcurve.</a>

    <p class="bookend">DONE: Analyzing telescope images. <b>Tutorial completed!</b></p>

    <!--<p>The data for the lightcurve you see here is at /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt. Click the folder icon in the left nav to navigate to it.-->

    <!--<div class="button">Download lightcurve data</div>-->

'''))

do_download = input("Download lightcurve data? (y/N) ")
if do_download in ['Y','y']:
  if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
    display(HTML('<p>Downloading now...</p>'))
    showProgress(2)
    files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
  else: 
    display(HTML('<p>Couldn\'t find output file. Alert the developers to ensure it is in the sample-data repo being used.</p>'))


# url for a file    https://drive.google.com/open?id=29PGh8XCts3mlMP6zRphvnIcbv27boawn



display(HTML('''
  <h2>Congratulations!</h2> 
  <h3>You have successfully generated a lightcurve showing the transit of HAT-P-32 b around HAT-P-32</h3>

  <p class="step">This exoplanet was actually discovered in 2011, and you can <a target="_blank" href="https://exoplanets.nasa.gov/exoplanet-catalog/1434/HAT-P-32-b/">view it in 3D on exoplanets.nasa.gov</a>.</p>

  <p class="step">If you downloaded the data to your hard drive (as "AAVSO_HAT-P-32 b_2017-12-19.txt"), it will be in a format suitable for submission to AAVSO, though please don't submit it! </p>
'''))

---

## End of Tutorial

<font size=2>
What would you like to do next?

<a href="https://exoplanets-5.client.mooreboeck.com/exoplanet-watch/exotic/">Try using EXOTIC with your own observation data</a>
</font>


<font size=2>
<a href="https://exoplanets-5.client.mooreboeck.com/exoplanet-watch/how-to-contribute/how-to-observe/">Learn how to observe exoplanets</a>

<a href="https://exoplanets.nasa.gov/exoplanet-watch/about-exoplanet-watch/contact-us/">Provide feedback for this tutorial</a>
</font>